In [ ]:
# Fetch all .po files from FE Workflows (for smart start) - Extract copy and create one giant .csv with the translations

In [202]:
# import libraries

#process webpages
from bs4 import BeautifulSoup

#interact with site
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException

from random import randint
import datetime
import os
import time
import pandas as pd
import numpy as np

import shutil
import glob

In [203]:
# set download options
options = Options();

options.set_preference("browser.download.folderList",1);
options.set_preference("browser.download.manager.showWhenStarting", False);
options.set_preference("browser.download.dir","/translation_data/");
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/octet-stream,application/vnd.ms-excel");

In [204]:
# check thst options have updated
options.preferences

{'browser.download.folderList': 1,
 'browser.download.manager.showWhenStarting': False,
 'browser.download.dir': '/translation_data/',
 'browser.helperApps.neverAsk.saveToDisk': 'application/octet-stream,application/vnd.ms-excel'}

In [205]:
# landing page
url = "http://fe.smartstart.prd.feersum.io/admin/workflow/workflow/"

In [206]:
# create a new Firefox session
driver = webdriver.Firefox(options=options);
# driver.implicitly_wait(30)
driver.get(url)

In [207]:
# Fill in login details
username = driver.find_element_by_id("id_username")
password = driver.find_element_by_id("id_password")

username.send_keys("devs")
password.send_keys("axvkcl4389xv%^3klj498dfg")

In [208]:
#Login
python_button = driver.find_element_by_class_name("btn ")
python_button.click() 

In [145]:
# Get page source for first page of interest
# main_soup=BeautifulSoup(driver.page_source, 'lxml')

In [146]:
# determine total number of workflows
# number_of_workflows = int(main_soup.find('div',class_="pagination-info muted").text.split()[-3])

In [147]:
# number_of_workflows

239

In [194]:
def filter_exportPO_links(main_soup):
    """filter export PO links on a page based on non-zero number of extracted phrases
    """
    links = []
    for row in main_soup.find_all('table', id='result_list'):
        for i,phrase_count in enumerate(row.find_all('td', class_='field-num_translatable_phrases')):
            if (int(phrase_count.string)) > 0:
                links.append(row.find_all(
                    'td',class_='field-translation_actions')[i].find_all(
                    'button', string="Export PO")[0].attrs['onclick'].split(
        "location.href=")[1].replace("'","").replace(
        '/translations','translations'))

    return links

In [180]:
# export_links = filter_exportPO_links(main_soup=main_soup)
# export_links

[]

In [426]:
# workflow_links = []
# for link in main_soup.find_all('button', string="Export PO"):
#         workflow_links.append(link)

In [427]:
# link_url = workflow_links[9].attrs['onclick'].split(
#     "location.href=")[1].replace("'","").replace(
#     '/translations','translations')

In [730]:
# link_url = export_links[0]

In [ ]:
# The first save has to be done outside the for loop

In [80]:
# select po file
# driver.get('http://fe.smartstart.prd.feersum.io/'+export_links[0])

In [195]:
target_langs = ['afrikaans','zul', 'xho',
                'southern sotho', 'sepedi' ,'tswana',
                'venda', 'tso', 'swat','south nde']

In [82]:
# lang_dropdown = driver.find_elements_by_class_name('select2-selection__arrow')

In [83]:
# lang_dropdown[1].click()

In [84]:
#  enter_lang = driver.find_element_by_class_name('select2-search__field')

In [85]:
# enter_lang.send_keys('afrik')

In [86]:
# enter_lang.send_keys(Keys.RETURN)

In [87]:
# export_button = driver.find_element_by_css_selector('#content > form:nth-child(2) > input:nth-child(4)')

In [88]:
# export_button.click()

In [90]:
# driver.back()

In [393]:

# driver.quit()

In [213]:
%%time
#link_url is a link to the po file, then download the 10 translations for each
# still need to iterate over all the pages
def extract_po_files(export_links,langs):
    """
    """
    for wf,link_url in enumerate(export_links):
        f'Working on workflow number {wf}'
        # select po file
        driver.get('http://fe.smartstart.prd.feersum.io/'+link_url)
        for lang in target_langs:
            f'{lang}'
            lang_dropdown = driver.find_elements_by_class_name('select2-selection__arrow')
            lang_dropdown[1].click()
            enter_lang = driver.find_element_by_class_name('select2-search__field')
            enter_lang.send_keys(lang)
            time.sleep(0.75)
            enter_lang.send_keys(Keys.RETURN)
            time.sleep(0.25)
            export_button = driver.find_element_by_css_selector('#content > form:nth-child(2) > input:nth-child(4)')
            export_button.click()
        f'Finsihed on page'

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


In [197]:
%%time
next_page_exists=True
page_index = 2
all_links = []
pages = []

while next_page_exists:
    try:
        # Get page source for first page of interest
        main_soup=BeautifulSoup(driver.page_source, 'lxml')
        export_links = filter_exportPO_links(main_soup=main_soup)
        
        all_links.append(export_links)
        pages.append(page_index)
        
        next_page = driver.find_element_by_class_name(
            'pagination').find_element_by_link_text(str(page_index))
        next_page.click()
        
        page_index+=1
        
        #     extract_po_files(export_links=export_links, langs = target_langs)
#         driver.back()
        
    except NoSuchElementException:
        print('No more pages to to scrape ... exiting now')
        next_page_exists=False
#     if page_index > 8:
#         break
        
print('The end')
driver.quit()   
    

No more pages to to scrape ... exiting now
The end
CPU times: user 881 ms, sys: 15.2 ms, total: 896 ms
Wall time: 11.3 s


In [198]:
links = [link for links in all_links for link in links ]

In [199]:
len(links), len(set(links))

(43, 43)

In [211]:
links

['translations/export_po?workflow=HXYx1RG4SE2APBy_RJ5gfw',
 'translations/export_po?workflow=wQL4TQy9TH-tvsGUTz_NMA',
 'translations/export_po?workflow=ZTokMsKQRP-6xojMrMzeRA',
 'translations/export_po?workflow=H1_mo9_5R5a7RIRLwA4TLQ',
 'translations/export_po?workflow=Ylvs54KpSr6MRrYt_DLc4g',
 'translations/export_po?workflow=P7ITtSftS7ik2I2FoWTD5g',
 'translations/export_po?workflow=f2P1dIHYR--WWY0gPbdZhA',
 'translations/export_po?workflow=Fukqk6TLRbq1dG6zLJELQw',
 'translations/export_po?workflow=FPGDUvRQR6eiQHYmnSWVCA',
 'translations/export_po?workflow=QQvBkyn0QqeGy7NV1aka9Q',
 'translations/export_po?workflow=KRbWcIz9SJqQSOxq5FxxbA',
 'translations/export_po?workflow=B3iVazNFRtyYwyn6c6owEg',
 'translations/export_po?workflow=rkxfsafHRFq5JEsQuR2OUw',
 'translations/export_po?workflow=yq1WX6bYQbeQVrWU6xGbOg',
 'translations/export_po?workflow=qi2_WFL7SMuQRWoMUdBAuA',
 'translations/export_po?workflow=bOnYFz2BRgixd6nGvpdeBg',
 'translations/export_po?workflow=Xqu76I1GTuGVWVnYG9p2HQ

In [ ]:
http://fe.smartstart.prd.feersum.io/translations/export_po?workflow=hzoId-qjSamZr22qQg8rfA

In [214]:
links[34:]

['translations/export_po?workflow=hzoId-qjSamZr22qQg8rfA',
 'translations/export_po?workflow=dqEbY089ReS8TexzEQbNlA',
 'translations/export_po?workflow=xvU22jkoRiOGpK4BVO4juA',
 'translations/export_po?workflow=p1sgHULbTnC2J8reOmaBgg',
 'translations/export_po?workflow=u_CFE22SQmySWzEyaPjAwg',
 'translations/export_po?workflow=u61hj3QiTl2kvod-yDqzgw',
 'translations/export_po?workflow=3SR9y5mIS9uPHMfvXCDYDA',
 'translations/export_po?workflow=flZcvRFWTOuSp85ZXbEDAQ',
 'translations/export_po?workflow=5jpuAlxiSC2qeZd7n-eztw']

In [215]:
%%time
extract_po_files(export_links=links[34:], langs = target_langs)

CPU times: user 975 ms, sys: 86.7 ms, total: 1.06 s
Wall time: 9min 58s


In [227]:
src_fldr = '/home/ari/Downloads/'
dest_fldr = '/home/ari/Documents/2018/Work/Praekelt/Repos/po-files/translations/'


files = os.listdir(src_fldr)

In [229]:
for po_file in glob.glob(src_fldr+"*.po"):
    shutil.copy2(po_file, dest_fldr);
    

In [217]:
driver.close()